# Methode target-IMSE

## Importation de donnees traitees

In [2]:
data<-read.csv("data_visco.csv")
data

vis,X__2,X__3,X__4,X__5,X__6,X__7,X__8,X__9,X__10,⋯,X__125,X__127,X__128,X__130,X__131,X__132,X__133,X__135,X__136,X__138
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2430,0.00,0,9.42,0,0,0,8.52,0,0,⋯,0,0,0,0,0,0,0,0,0.00,0.00
4433,0.00,0,9.42,0,0,0,8.53,0,0,⋯,0,0,0,0,0,0,0,0,0.00,0.00
25,0.39,0,3.58,0,0,0,14.25,0,0,⋯,0,0,0,0,0,0,0,0,3.56,1.34
17,0.00,0,3.33,0,0,0,14.84,0,0,⋯,0,0,0,0,0,0,0,0,0.00,1.24
21,0.00,0,3.81,0,0,0,14.08,0,0,⋯,0,0,0,0,0,0,0,0,3.31,1.24
64,0.00,0,0.00,0,0,0,0.00,0,0,⋯,0,0,0,0,0,0,0,0,0.00,0.00
79,0.00,0,3.00,0,0,0,18.00,0,0,⋯,0,0,0,0,0,0,0,0,1.00,2.00
5864,0.00,0,9.43,0,0,0,8.53,0,0,⋯,0,0,0,0,0,0,0,0,0.00,0.00
946,0.00,0,0.00,0,0,0,0.00,0,0,⋯,0,0,0,0,0,0,0,0,0.00,0.00


In [3]:
library("KrigInv")

Le chargement a nécessité le package : DiceKriging



In [4]:
x = data[1:60,1:116]
#x = matrix(sample(5:500, 116*30, replace=TRUE),nrow = 30)
data_reduced = data[1:120,1:116]
#des = data_reduced[c(-1)]
shape = c(dim(data_reduced))
noise = matrix(runif(prod(shape),min = 0.00000001,max = 0.00000002), nrow = 120)
mod = km(formula = ~1, design = data_reduced + noise, response = data_reduced[1])
tmse_optim(x, model = mod, T = 30)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  12076 0.78 1.966741e-08 44.86 1.99

[1] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
 [9] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
[17] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
[25] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
[33] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
[41] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
[49] 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323 334.4323
[57] 334.4323 334.4323 334.4323 334.4323

In [7]:
fn<-function(x){
  tmse_optim(x, model = mod, T = 30)
}
contr<-function(x){
  h<-numeric(2)
  h[1]<- 100-2*x #g(x)>=0
  h[2]<- x  #-g(x)>=0
  return(h)
}
library("nloptr")
cobyla(x,fn,hin=contr)

For consistency with the rest of the package the inequality sign may be switched from >= to <= in a future nloptr version.

Warning message in h[1] <- 100 - 2 * x:
“le nombre d'objets à remplacer n'est pas multiple de la taille du remplacement”
Warning message in h[2] <- x:
“le nombre d'objets à remplacer n'est pas multiple de la taille du remplacement”


ERROR: Error in (-1) * f2(x, ...): argument non numérique pour un opérateur binaire


In [9]:
simul <- function(d)
{
  # set.seed(18)
  u = runif(n=d, min=0,max=1)
  #replicate(n=5,u) si on doit tester avec plusieurs vecteurs diff
  u = c(0,u,1)
  u = sort(u)
  x = u[2:(d+2)] - u[1:(d+1)]  
  return(x)
}

g = function (x, model, T, method.param = NULL) 
{
    y <- t(x)
    if (ncol(y) == model@d) 
        z <- y
    if (ncol(y) != model@d) 
        z <- x
    krig <- predict_nobias_km(object = model, newdata = as.data.frame(z), 
        type = "UK", se.compute = TRUE)
    mk <- krig$mean
    sk <- krig$sd
    if (is.null(method.param)) 
        method.param <- 0
    epsilon <- method.param
    if (length(T) == 1) {
        W <- 1/sqrt(2 * pi * (sk^2 + epsilon^2)) * exp(-0.5 * 
            ((mk - T)/sqrt(sk^2 + epsilon^2))^2)
    }
    else {
        W0 <- 1/sqrt(2 * pi * (sk^2 + epsilon^2))
        W <- 0
        for (i in 1:length(T)) {
            Ti <- T[i]
            W <- W + W0 * exp(-0.5 * ((mk - Ti)/sqrt(sk^2 + epsilon^2))^2)
        }
    }
    g <- W * sk^2
    return(g)
}


In [10]:
data_Lasso = read.csv("data_lasso.csv")
d = 56
N = 10*d
s = 0
data_Lasso_reduced = data_Lasso[1:N,]
shape = c(dim(data_Lasso_reduced))
noise = matrix(runif(prod(shape),min = 0.0001,max = 0.0002), nrow = N)
mod = km(formula = ~1, design = data_Lasso_reduced + noise, response = data_Lasso_reduced$vis)
for(i in 0:N){
  x = simul(d) 
  s = s+ g(x, model = mod,T=400)
}
tmse = s/N


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  2.286692e+12 0.0001996054 57.00012 149.5401 43.94008 0.0001990512 70.00013 0.6001537 121.9601 0.0001994942 117.8001 0.08018567 119.04 80.00008 57.84017 28.52014 45.30003 0.0001993278 110.7201 0.0001993602 113.7602 0.0001993871 158.7399 0.1600249 0.9000968 59.92019 0.4801327 49.78005 157.64 
  - best initial criterion value(s) :  -14072.1 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=        14072  |proj g|=       2.9839
At iterate     1  f =        13507  |proj g|=        3.7695
At it

ERROR: Error in predict_nobias_km(object = model, newdata = as.data.frame(z), : newdata must have the same numbers of columns ( 1 ) than the experimental design ( 29 )
